In [35]:
from brayns import Client
from circuitexplorer import CircuitExplorer
brayns = Client('localhost:5000')
ce = CircuitExplorer(brayns)

In [36]:
config = '/home/favreau/git/install/release/share/BBPTestData/circuitBuilding_1000neurons/BlueConfig'

In [37]:
model = ce.load_circuit(
    use_sdf_soma=True, use_sdf_branches=False, use_sdf_nucleus=True,
    load_soma=True, load_axon=True, load_apical_dendrite=True, load_dendrite=True,
    load_afferent_synapses=True, load_efferent_synapses=True,
    circuit_color_scheme=ce.CIRCUIT_COLOR_SCHEME_NEURON_BY_ID,
    morphology_color_scheme=ce.MORPHOLOGY_COLOR_SCHEME_NONE,
    density=100, path=config, generate_internals=True,
    targets=['mini50']
)

In [38]:
status = brayns.set_camera(
    orientation=[-0.007149015377189473, -0.3525297158187466, -0.006351682723416716, 0.935751755098793],
    position=[-7.259142462546265, 1336.4928115590612, 113.62684206008949],
    target=[39.16958356316163, 1335.2359535608118, 60.74596607987225]
)

In [39]:
def set_materials(model_id, palette_name, opacity=1.0):
    
    simulation_data_casts=list()
    opacities=list()
    refraction_indices=list()
    reflection_indices=list()
    shading_modes=list()
    diffuse_colors=list()
    specular_colors=list()
    specular_exponents=list()
    material_ids=list()
    glossinesses=list()
    emissions=list()
    clips=list()
    user_parameters=list()
    
    import seaborn as sns
    material_ids = ce.get_material_ids(model_id)['ids']
    nb_materials = len(material_ids)
    palette = sns.color_palette(palette_name, nb_materials)
    
    for i in range(nb_materials):
        c=palette[i]
        material_id = material_ids[i] % ce.NB_MATERIALS_PER_MORPHOLOGY
        if material_id in [ce.MATERIAL_OFFSET_MITOCHONDRION, ce.MATERIAL_OFFSET_NUCLEUS]:
            opacities.append(1.0)
            shading_modes.append(ce.SHADING_MODE_PERLIN)
            user_parameters.append(0.1)
            specular_exponents.append(5)
            glossinesses.append(0.1)
            if material_id == ce.MATERIAL_OFFSET_MITOCHONDRION:
                emissions.append(0.0)
                diffuse_colors.append([0.4, 0.0, 0.4])
                specular_colors.append([0.4, 0.0, 0.4])
            else:
                emissions.append(0.0)
                diffuse_colors.append([1,1,1])
                specular_colors.append([1,1,1])
        elif material_id in [ce.MATERIAL_OFFSET_AFFERENT_SYNPASE, ce.MATERIAL_OFFSET_EFFERENT_SYNPASE]:
            # Synapses
            opacities.append(1.0)
            emissions.append(0.5)
            shading_modes.append(ce.SHADING_MODE_PERLIN)
            user_parameters.append(0.5)
            specular_exponents.append(5)
            glossinesses.append(0.2)
            if material_id == ce.MATERIAL_OFFSET_AFFERENT_SYNPASE:
                diffuse_colors.append([1.0, 0.5, 0.2])
                specular_colors.append([1.0, 0.5, 0.2])
            else:
                diffuse_colors.append([0.2, 0.5, 1.0])
                specular_colors.append([0.2, 0.5, 1.0])
        else:
            # Neuron
            opacities.append(opacity)
            emissions.append(0.0)
            shading_modes.append(ce.SHADING_MODE_PERLIN)
            user_parameters.append(0.2)
            glossinesses.append(0.5)
            specular_exponents.append(5.0)
            diffuse_colors.append(c)
            specular_colors.append(c)
            
        refraction_indices.append(1.1)
        reflection_indices.append(0.0)
        simulation_data_casts.append(False)
        clips.append(False)
        
    ce.set_material_extra_attributes(model_id)
    ce.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        simulation_data_casts=simulation_data_casts,
        opacities=opacities, reflection_indices=reflection_indices,
        shading_modes=shading_modes, user_parameters=user_parameters,
        diffuse_colors=diffuse_colors, specular_colors=specular_colors,
        specular_exponents=specular_exponents, glossinesses=glossinesses,
        emissions=emissions, refraction_indices=refraction_indices)
    
model_id = model['id']
ce.set_material_extra_attributes(model_id)
set_materials(model_id, 'Set3', 0.6)
status =  brayns.set_renderer()

In [40]:
status = brayns.set_renderer(current='circuit_explorer_advanced',
    subsampling=4, max_accum_frames=64)
params = brayns.CircuitExplorerAdvancedRendererParams()
params.exposure = 1.5
params.max_bounces = 10
params.epsilon_factor = 400.0
params.fog_start = 40.0
params.fog_thickness = 200.0
params.gi_samples = 0
params.gi_weight = 0.5
params.gi_distance = 200.0
params.shadows = 0.0
params.soft_shadows = 0.1
status = brayns.set_renderer_params(params)